# Contenido
* [Introducción](#intro)
* [Descripción del proyecto](#desc)
* [Análisis exploratorio](#exp)

## Introducción<a id='intro'></a>

Nuestro dataset a consta información acerca de autos usados y nuevos en venta de una compañia en un portal de internet. El objetivo del trabajo es realizar un pequeño preprocesamiento de la información y la realización de un análisis exploratorio, así como la creación de una API WEB desplegado en un servicio en la nube accesible para cualquier persona


 los datos presnetados son:

* price
* model_year
* model
* condition
* cylinders
* fuel — gasolina, diesel, etc.
* odometer — el millaje del vehículo cuando el anuncio fue publicado
* transmission
* paint_color
* is_4wd — si el vehículo tiene tracción a las 4 ruedas 
* date_posted — la fecha en la que el anuncio fue publicado
* days_listed — desde la publicación hasta que se elimina.

## Descripción del proyecto<a id= 'desc'></a>

In [3]:
# Libraries
import pandas as pd
import plotly.express as px
import streamlit as st

In [9]:
# read cvs file
df = pd.read_csv('C:/Users/jfcom/Documents/modulo_adicional/proyecto/vehicles_EDA/vehicles_us.csv')

# Show a data resume
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


Contamos con un dataset de 51525 datos con 13 columnas en algunas de ellas tenemos valores nulos. Los datos presnetados son:

* price: Precio en USD
* model_year: Año del modelo
* model: Modelo
* condition: Condición (new, like new, excellent, good, salvage, fair)
* cylinders: Número de cilíndros
* fuel: Tipo de combustible utilizado(gasolina, diesel, etc.)
* odometer: El millaje del vehículo cuando el anuncio fue publicado
* transmission: Tipo de transmisión
* paint_color: Color del auto
* is_4wd: Si el vehículo tiene tracción a las 4 ruedas 
* date_posted: Fecha en la que el anuncio fue publicado
* days_listed: días de la publicación desde el inicio hasta la eliminación.

A continuación mostraremos una parte de los datos y la cantidad de valores nulos

In [12]:
# Sample of data
df.head(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28
5,14990,2014.0,chrysler 300,excellent,6.0,gas,57954.0,automatic,sedan,black,1.0,2018-06-20,15
6,12990,2015.0,toyota camry,excellent,4.0,gas,79212.0,automatic,sedan,white,NaN,2018-12-27,73
7,15990,2013.0,honda pilot,excellent,6.0,gas,109473.0,automatic,SUV,black,1.0,2019-01-07,68
8,11500,2012.0,kia sorento,excellent,4.0,gas,104174.0,automatic,SUV,NaN,1.0,2018-07-16,19
9,9200,2008.0,honda pilot,excellent,NaN,gas,147191.0,automatic,SUV,blue,1.0,2019-02-15,17


In [6]:
df.isna().sum()

price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64

## Análisis exploratorio<a id = 'exp'></a>